# Teoria Moderna do Portfólio em Python
 _Baseado [nesse artigo do Medium](https://medium.com/turing-talks/teoria-moderna-do-portf%C3%B3lio-em-python-e8bad41ebb8b)_
 
## Introdução
A Teoria de Portfólio de Markowitz, desenvolvida pelo economista Harry Markowitz, é um amplamente utilizado para otimizar a alocação de ativos financeiros em um portfólio. Essa teoria leva em consideração o equilíbrio entre o **retorno esperado** e o **risco** associado a cada ativo, com o objetivo de encontrar a combinação ideal de ativos que ofereça o máximo de retorno para um determinado nível de risco ou o mínimo de risco para um determinado nível de retorno.

Nesse notebook serão desenvolvidos alguns exemplos de montagens de portfólio com base na Teoria de Markowitz.


## Exemplo 1 - Setor de Beleza/Estética

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import yfinance as yf
import turingquant as tq
import plotly.io as pio

In [16]:
#Foram escolhidas 5 empresas do ramo de beleza/estética para compor o portfólio (Estée Lauder Companies Inc. (EL);
#L'Oréal SA (OR.PA); The Procter & Gamble Company (PG); Ulta Beauty, Inc. (ULTA))
tickers = ['EL', 'OR.PA', 'PG', 'ULTA']

In [17]:
df_close = yf.download(tickers, start='2019-01-01', end='2022-12-31')['Close']

[*********************100%***********************]  4 of 4 completed


In [18]:
df_close

,EL,OR.PA,PG,ULTA
Date,,,,
2019-01-02,131.940002,199.600006,91.279999,247.970001
2019-01-03,125.680000,197.199997,90.639999,243.360001
2019-01-04,129.919998,200.000000,92.489998,255.029999
2019-01-07,131.500000,197.449997,92.120003,271.000000
2019-01-08,132.729996,198.649994,92.459999,276.000000
...,...,...,...,...
2022-12-23,241.020004,333.250000,152.619995,456.399994
2022-12-27,248.289993,336.750000,153.949997,464.000000
2022-12-28,246.190002,335.399994,151.960007,463.519989


## Passo-a-Passo:

**1. Escolha do Método:** são 3 os métodos possíveis - o que minimiza a volatilidade, o que maximiza o retorno e o que maximiza o sharpe ratio. Esse sharpe ratio pode ser optido pela fórmula $Sharpe$ $Ratio$ $=$ $(Retorno - RiskFree)$ $/$ $Volatilidade$. No modelo de Markowitz do Turing Quant, define-se que risk free = 0 (tudo tem risco), logo $Sharpe$ $Ratio$ $=$ $Retorno$ $/$ $Volatilidade$

**2. Encontro do Melhor Portfólio:** o melhor portfólio será aquele que, feitas todas as combinações de percentuais de valor investido para cada ação, gerar o melhor valor para a métrica em questão (min volatilidade, max retorno ou max sharpe_ratio)

**3. Visualização da Fronteira Eficiente:** feitas todas as possíveis combinações, também é possível plotá-las para entender melhor onde estão as melhores e piores soluções de um modo geral, não apenas A melhor solução

In [29]:
class Markowitz:
    def __init__(self, df_close, num_portfolios = 10000, risk_free = 0):
        self.df = df_close
        self.num_portfolios = num_portfolios
        self.risk_free = risk_free
        self.wallets = self._generate_wallets()
    
    def _generate_wallets(self):
        '''
        Gera carteiras com pesos aleatórios.
        Returns:
            wallets (dict): dicionário contendo os valores 'weights', 'returns', 'vol' e 'sharpe_ratio'
                            de todos os portfólios gerados 
        '''
        # vetores de dados
        portfolio_weights = []
        portfolio_exp_returns = []
        portfolio_vol = []
        portfolio_sharpe = []
        
        # retorno simples 
        r = self.df.pct_change()
        mean_returns = r.mean() * 252
        
        # matriz de covariância 
        covariance = np.cov(r[1:].T)

        for i in range(self.num_portfolios):
            # gerando pesos aleatórios
            k = np.random.rand(len(self.df.columns))
            w = k / sum (k) # normaliza o vetor 

            # retorno
            R = np.dot(mean_returns, w)

            # risco
            vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)

            # sharpe ratio
            sharpe = (R - self.risk_free)/vol

            portfolio_weights.append(w)
            portfolio_exp_returns.append(R)
            portfolio_vol.append(vol)
            portfolio_sharpe.append(sharpe)

        # métricas (colunas) de cada portfólio (linhas)
        metrics = pd.DataFrame({
            'returns': portfolio_exp_returns,
            'vol': portfolio_vol,
            'sharpe': portfolio_sharpe
        })

        # pesos de cada ativo (colunas) por portfólio (linhas)        
        weights = pd.DataFrame(portfolio_weights, columns=self.df.columns)

        # carteira = métricas + colunas com o peso de cada ativo
        wallets = pd.concat([metrics, weights], axis=1)
    
        return wallets
    
    def plot_efficient_frontier(self, method = 'sharpe_ratio'):
        '''
        Plota gráfico com a fronteira eficiente dos portfólios gerados. 
        
        Args: 
            method (string): Método utilizado para indicar o melhor portfólio
                            'sharpe_ratio' - Portfólio com melhor Sharpe ratio
                            'volatility' - Portfólio com menor volatilidade
                            'return' - Portfólio com maior retorno
        '''

        vol = self.wallets['vol']
        returns = self.wallets['returns']
        sharpe = self.wallets['sharpe']
        
        if method == 'sharpe_ratio':            
            best_port_idx = np.array(sharpe).argmax()

        elif method == 'volatility':            
            best_port_idx = np.array(vol).argmin()

        elif method == 'return':             
            best_port_idx = np.array(returns).argmax()

        else:
            raise ValueError(
                f"method espera 'sharpe_ratio', 'volatility' ou 'return', não '{method}'"
            )

        y_axis = returns[best_port_idx]
        X_axis = vol[best_port_idx]

        # Plota todos os portfólios
        fig = px.scatter(
            self.wallets,
            x='vol',
            y='returns',
            hover_data=self.df.columns,
            color='sharpe'
        )

        # Customizações gerais do gráfico
        fig.update_layout(
            width=600, height=600,
            margin=dict(l=10, r=10, t=50, b=10),
            title='Efficient Frontier',
            xaxis_title="Volatility",
            yaxis_title="Returns",
        )

        # Exibe o ponto do melhor portfólio em vermelho
        fig.update_traces(
            marker=dict(size=9, opacity=0.6),
            selectedpoints=[best_port_idx],
            selected=dict(marker=dict(color='black', opacity=1))
        )

        fig.show()
        
        # funçao best_portfolio
    def best_portfolio(self, method = 'sharpe_ratio'):
                
        vol = self.wallets['vol']
        returns = self.wallets['returns']
        sharpe = self.wallets['sharpe']
        weights = self.wallets.iloc[:, 3:]
            
        if method == 'sharpe_ratio':
                best_port_idx = np.array(sharpe).argmax()

        elif method == 'volatility':
                best_port_idx = np.array(vol).argmin()

        elif method == 'return':
                best_port_idx = np.array(returns).argmax()

        else:
            raise ValueError(
                f"method espera 'sharpe_ratio', 'volatility' ou 'return', não '{method}'"
            )

        return weights.iloc[best_port_idx]
        

In [20]:
markowitz = Markowitz(df_close)

Note que assim que criamos um obeto da classe Markowitz, chamamos o método _generate_wallets para gerar 10000 carteiras com pesos aleatórios, e calcular o sharpe, volatilidade e retorno esperado (média ponderada dos retornos ateriores) de cada uma. Esses dados são guardados como atributo do objeto, e serão acessados por outros métodos como o plot_efficient_frontier. Ou seja, a quanto mais carteiras geradas, maior a chance de se aproximar do que seria uma carteira com os pesos ótimos.

In [35]:
fronteira = markowitz.plot_efficient_frontier(method='sharpe_ratio')

TypeError: Markowitz.plot_efficient_frontier() got multiple values for argument 'method'

                        PONTO DO MELHOR PORTFÓLIO EM PRETO

### Encontrando os melhores pesos para o portfólio

O peso de um ativo em uma carteira representa uma porcentagem do valor total dos investimentos. Com os pesos de um portfólio, pode-se ter uma visão holística de como está disposta sua composição, sendo fácil de visualizar os ativos que influem mais ou menos na carteira. 

Utilizando a função **best_portfolio**, podemos encontrar os pesos do melhor portfólio formado pelas empresas de beleza em questão ('EL', 'OR.PA', 'PG', 'ULTA'). Vale ressaltar que podemos indicar três tipos métodos para a análise: 
- **sharpe_ratio**: portfólio com o melhor *sharpe ratio*;
- **volatility**: portfólio com a menor volatilidade;
- **return**: portfólio com o maior retorno;

Nesse caso, estamos utilizando o método de melhor *sharpe ratio*. 

In [34]:
melhor_portfolio = Markowitz.best_portfolio(markowitz, method = 'sharpe_ratio')
print(melhor_portfolio)

EL       0.063224
OR.PA    0.321405
PG       0.454852
ULTA     0.160519
Name: 5189, dtype: float64


A função resulta nos valores: 

|  ativo|  peso |
|-------|-------|
|   EL  |   0.063224    |
| OR.PA |   0.321405    |
|   PG  |   0.321405    |
|  ULTA |   0.160519    |

Eles espelham que o portfólio que maximiza o *sharpe ratio* é composto majoritariamente por ativos da 'OR.PA' e da 'PG', que juntos somam mais de 50% dos pesos da carteira. Além disso, é perceptível que [cont...]

**Observações:**
1. Nota-se que, dentro do período analisado (2019-2022), as diferentes combinações apresentam retornos muito próximos (quase todos na faixa entre 0.16 e 0.24), enquanto a volatilidade tem uma variação muito maior (0.2 até 0.4)
2. Nesse sentido, usando o modelo do sharpe ratio, o melhor portfólio foi uma combinação que preferenciou a volatilidade ao retorno. Em outras palavras, ainda que tenha tido um retorno abaixo da média, foi a melhor solução devido à baixíssima volatilidade
3. No exemplo acima, o melhor portfólio foi um que se baseou principalmente na Procter & Gamble e na L'Oréal, mas ainda contou com participações significativas (>5%) dos outros dois ativos, o que pode ter contribuído para a baixa volatilidade

## Exemplo 2 - Big Techs (Sharpe Ratio pode ser ruim?)

In [8]:
#Foram selecionadas as 4 big techs: Apple, Amazon, Google e Microsoft
tickers_2 = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
df_close_2 = yf.download(tickers_2, start='2022-01-01', end='2022-12-31')['Close']

[*********************100%***********************]  4 of 4 completed


In [9]:
markowitz_2 = Markowitz(df_close_2)
fronteira_2 = markowitz_2.plot_efficient_frontier(method='sharpe_ratio')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Observações:**
1. Nesse caso, os retornos são negativo. Com isso, pela fórmula $Sharpe$ $Ratio$ $=$ $Retorno$ $/$ $Volatilidade$, uma volatilidade maior acaba por gerar um sharpe ratio artificialmente maior, levando a conclusões equivocadas. 
2. Por isso, no exemplo acima, a "melhor solução" apresenta maior volatilidade e retornos mais negativos (menores, portanto) que outras soluções, mostrando a problemática de usar esse tipo de método para retornos negativos